In [ ]:
import polars as pl
import numpy as np
import os
import matplotlib.pyplot as plt
import emd
from dtw import *
from utils import *
# from dp.dp_utils import compute_all_costs

time = []
fs = []
hr = []
mx = []

# Recordings files paths
path = os.getcwd() + r"\data_healthy"
file_paths = sorted([os.path.join(path, file) for file in os.listdir(path)], key=extract_recording_number)

# Read data from the files
for file_path in file_paths:
    df = pl.read_csv(file_path, separator=";", decimal_comma=True)
    t_hat, _ = convert_datetime_to_time(df["DateTime"].to_numpy())
    time.append(t_hat)
    hr.append(df["hr"].to_numpy())
    if df["FVL"].mean() > df["FVR"].mean():
        mx.append(df["mx_l"].to_numpy())
    else:
        mx.append(df["mx_r"].to_numpy())

In [ ]:
for record_number in range(len(mx)):
    imf_mx = emd.sift.sift(mx[record_number])
    n = imf_mx.shape[1]
    imf_sum = imf_mx[:, n - 3] + imf_mx[:, n - 2] + imf_mx[:, n - 1]
    alignment = dtw(imf_sum, hr[record_number], keep_internals=False, distance_only=False)
    normalized_distance = alignment.distance / (len(hr[record_number]) + len(imf_mx))
    

In [ ]:
for file_number in range(len(mx)):
    imf_hr = emd.sift.sift(hr[file_number])
    # emd.plotting.plot_imfs(imf_hr, time_vect=time[file_number], xlabel="Time [s]")
    n = imf_hr.shape[1]
    imf_hr = imf_hr[:, n - 3] + imf_hr[:, n - 2] + imf_hr[:, n - 1]